# 📝 Exercise M6.01

The aim of this notebook is to investigate if we can tune the hyperparameters
of a bagging regressor and evaluate the gain obtained.

We will load the California housing dataset and split it into a training and
a testing set.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

data, target = fetch_california_housing(as_frame=True, return_X_y=True)
target *= 100  # rescale the target in k$
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=0, test_size=0.5)

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">If you want a deeper overview regarding this dataset, you can refer to the
Appendix - Datasets description section at the end of this MOOC.</p>
</div>

Create a `BaggingRegressor` and provide a `DecisionTreeRegressor`
to its parameter `base_estimator`. Train the regressor and evaluate its
generalization performance on the testing set using the mean absolute error.

In [10]:
# Write your code here.

from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor


bagged_trees = BaggingRegressor(
    base_estimator=DecisionTreeRegressor(),
    n_estimators=100,
)
bagged_trees.fit(data_train, target_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=100)

In [11]:
from sklearn.metrics import mean_absolute_error as MAE
y_hat = bagged_trees.predict(data_test)
MAE(y_hat, target_test)


34.626424218023274

Now, create a `RandomizedSearchCV` instance using the previous model and
tune the important parameters of the bagging regressor. Find the best
parameters  and check if you are able to find a set of parameters that
improve the default regressor still using the mean absolute error as a
metric.

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">You can list the bagging regressor's parameters using the <tt class="docutils literal">get_params</tt>
method.</p>
</div>

In [12]:
# Write your code here.
# solution
for param in bagged_trees.get_params().keys():
    print(param)

base_estimator__ccp_alpha
base_estimator__criterion
base_estimator__max_depth
base_estimator__max_features
base_estimator__max_leaf_nodes
base_estimator__min_impurity_decrease
base_estimator__min_samples_leaf
base_estimator__min_samples_split
base_estimator__min_weight_fraction_leaf
base_estimator__random_state
base_estimator__splitter
base_estimator
bootstrap
bootstrap_features
max_features
max_samples
n_estimators
n_jobs
oob_score
random_state
verbose
warm_start


In [14]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "n_estimators": randint(10, 30),
    "max_samples": [0.5, 0.8, 1.0],
    "max_features": [0.5, 0.8, 1.0],
    "base_estimator__max_depth": randint(3, 10),
}
search = RandomizedSearchCV(
    bagged_trees, param_grid, n_iter=20, scoring="neg_mean_absolute_error"
)
_ = search.fit(data_train, target_train)

In [15]:
import pandas as pd

columns = [f"param_{name}" for name in param_grid.keys()]
columns += ["mean_test_error", "std_test_error"]
cv_results = pd.DataFrame(search.cv_results_)
cv_results["mean_test_error"] = -cv_results["mean_test_score"]
cv_results["std_test_error"] = cv_results["std_test_score"]
cv_results[columns].sort_values(by="mean_test_error")

,param_n_estimators,param_max_samples,param_max_features,param_base_estimator__max_depth,mean_test_error,std_test_error
7,27,1.0,1.0,9,39.154961,1.231822
0,12,0.5,0.8,9,39.613596,1.356890
6,16,0.5,0.8,9,40.465315,1.196803
10,22,0.5,0.8,8,40.893247,1.570620
19,28,0.5,0.8,6,44.849582,1.078704
18,28,1.0,0.8,6,44.871792,0.969461
2,23,0.5,1.0,6,44.997904,1.430693
4,28,0.5,0.5,8,45.324578,1.330292
13,11,0.5,0.8,6,45.543463,1.062482
14,18,1.0,0.5,7,47.031426,2.106575


In [17]:
target_predicted = search.predict(data_test)
print(f"Mean absolute error after tuning of the bagging regressor:\n"
      f"{MAE(target_test, target_predicted):.2f} k$")

Mean absolute error after tuning of the bagging regressor:
39.19 k$
